In [3]:
import tweepy
import csv
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
####input your credentials here
consumer_key = '95cMtk1vJvEEW2rlMR0kIU9lE'
consumer_secret = 'pMQFi7LBdcudKDNZOokUJGS8mDxQanUv8spxBDdTLiwSZBuUOM'
access_token = '1036313393114767360-BZ8Qpi02ghRvehhcITEIyl7SmGWmU6'
access_token_secret = 'C7VAqGDhTdB424iBtEwF1CJI9YPTcvNvLjFmaCXENNv3G'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
#####United Airlines
# Open/Create a file to append data
csvFile = open('ua.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for status in tweepy.Cursor(api.search,q="#blacklivesmatter",count=1,tweet_mode = 'extended',
                           lang="en",
                           since="2017-04-03").items():
    print("Tweet Startd")
    if hasattr(status, "retweeted_status"):  # Check if Retweet
        try:
            print(status.retweeted_status.extended_tweet["full_text"])
        except AttributeError:
            print(status.retweeted_status.full_text)
    else:
        try:
            print(status.extended_tweet["full_text"])
        except AttributeError:
            print(status.full_text)
    #print(status.parse())
    print("Tweet Ended")
    #csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

Tweet Startd
https://t.co/XWFUWbD9bm
It must be said that Morey and the Americans never dreamed that the people he supported supported racial discrimination.
#NBA #HongKonazi #BlackLivesMatter 
#HongKong #LeBronJames https://t.co/72MtrAMtpF


TypeError: parse() missing 2 required positional arguments: 'api' and 'json'

In [8]:
class Tweet():
    
    def __init__(self, status_response):
        self.id = status_response.id
        self._get_text(status_response)
        self.user_info = status_response.user
        self.entitites = status_response.entities
        
    def _get_text(self, status):
        if hasattr(status, "retweeted_status"):  # Check if Retweet
            try:
                self.text = status.retweeted_status.extended_tweet["full_text"]
            except AttributeError:
                self.text = status.retweeted_status.full_text
        else:
            try:
                self.text = status.extended_tweet["full_text"]
            except AttributeError:
                self.text = status.full_text
    
    #code for generating named entities of each tweet
    def _get_named_entities(self):
        pass
    #code for generating event phrases
    def _get_event_phrases(self):
        pass
    
    def get_graph_entities(self):
        return self._get_named_entities() + self._get_event_phrases()
    
    def __hash__(self):
        return self.id
    def __eq__(self, other):
        return self.id == other.id
    
class TweetRetriever():

    def __init__(self):
        consumer_key = '95cMtk1vJvEEW2rlMR0kIU9lE'
        consumer_secret = 'pMQFi7LBdcudKDNZOokUJGS8mDxQanUv8spxBDdTLiwSZBuUOM'
        access_token = '1036313393114767360-BZ8Qpi02ghRvehhcITEIyl7SmGWmU6'
        access_token_secret = 'C7VAqGDhTdB424iBtEwF1CJI9YPTcvNvLjFmaCXENNv3G'
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        self.api = tweepy.API(auth,wait_on_rate_limit=True)
    
    def getTweets(self, hashtag, count = 10):
        tweets = []
        for status in tweepy.Cursor(self.api.search, q = hashtag, count = count, 
                                    tweet_mode = 'extended', lang = 'en').items():
            tweets.append(Tweet(status))
        return tweets

    

In [10]:
class GraphNode():
    
    def __init__(self, name):
        self.name = name
        self.tweeets = set()
        self.value = 0
        
    def add_tweet(self, tweet):
        self.tweeets.add(tweet)
    
    def common_tweets(self, other):
        return len(self.tweeets.intersection(other.tweets))
    
    def __hash__(self):
        return self.name
    def __eq__(self, other):
        return self.name == other.name
    
class TweetGraph():
    
    def __init__(self, topic):
        self.topic = topic
        self.nodes = {}
        self.edge_map = {}
        
    def add_entity(self, name, tweet_ref):
        if name not in nodes:
            nodes[name] = GraphNode(name)
        nodes[name].add_tweet(tweet_ref)
    
    def add_edge(self, node1, node2):
        assert node1.name in self.node_map
        assert node2.name in self.node_map
        weight = node1.common_tweets(node2)
        self.edge_map.setdefault(node1.name, {}).setdefault(node2.name, weight)
        self.edge_map.setdefault(node2.name, {}).setdefault(node1.name, weight)
    
    def compute_all_edges(self):
        for node1 in self.nodes.values():
            for node2 in self.nodes.values():
                self.add_edge(node1, node2)
    
    def _get_pagerank_matrix(self):
        x = [[0 for _ in len(self.nodes)] for _ in len(self.nodes)]
        for i, node1 in enumerate(self.nodes.values()):
            wsum = 0
            for node2 in self.nodes.values():
                wsum += self.edge_map.get(node1.name, {}).get(node2.name, 0)
            for j, node2 in enumerate(self.nodes.values()):
                x[i][j] = self.edge_map.get(node1.name, {}).get(node2.name, 0)/wsum
        return np.array(x)
    
    def set_textrank_values(self, d = 0.85):
        rank_graph = nx.from_numpy_array(self._get_pagerank_matrix())
        node_scores = nx.pagerank(rank_graph, aplpha = d)
        for i, node in enumerate(self.nodes.values()):
            node.value = node_scores[i]
    
    def get_weight(self, node1, node2):
        return self.edge_map(node1.name, {}).get(node2.name, 0)
    
    def get_topic_similarity(self, node):
        if node.name in self.topic:
            return len(node.tweeets)
        return 1
    
    def get_nodes_above_thres(self, thres = 1):
        nodes = []
        value_sum = 0
        for node in self.nodes.values():
            if node.value > thres:
                nodes.append(node)
                value_sum += node.value
        return nodes, value_sum
        

In [12]:
def createGraph(topic, tweets):
    tweetGraph = TweetGraph(topic)
    for tweet in tweets:
        graph_entities = tweet.get_graph_entities()
        for name in graph_entities:
            tweetGraph.add_entity(name, tweet)
    tweetGraph.compute_all_edges()
    tweetGraph.set_textrank_values()
    return tweetGraph

def partitionGraph(tweetGraph, alpha, beta, high_rank_thres = 1):
    
    #initialize highly ranked nodes and their total values sum
    high_ranked_nodes, total_value_sum = tweetGraph.get_nodes_above_thres(high_rank_thres)
    high_ranked_nodes = sorted(high_ranked_nodes, key = lambda x: x.value)
    partitions = []
    
    #partitioning loop
    while len(high_ranked_nodes):
        #entity set is the nodes in the partition
        entity_set = set()
        repr_node = high_ranked_nodes.pop()
        entitiy_set.add(repr_node)
        repr_node_topic_similarity = tweetGraph.get_topic_similarity(repr_node)
        value_sum = repr_node.value
        
        for node in high_ranked_nodes:
            node_edge_weight = tweetGraph.get_weight(repr_node, node)
            node_topic_similarity = tweetGraph.get_topic_similarity(node)
            
            if node_edge_weight/repr_node_topic_similarity > alpha and \
                node_topic_similarity/repr_node_topic_similarity > alpha:
                entity_set.add(node)
                value_sum += node.value
        
        if value_sum/total_value_sum > beta:
            temp = []
            for node in high_ranked_nodes:
                if node not in entitiy_set:
                    temp.append(node)
            high_ranked_nodes = temp
            
        partitions.append([])
        for node in list(entity_set):
            partitions[-1].append(node)
            
    return partitions
    

In [5]:
ret = TweetRetriever()

In [13]:
tweets = ret.getTweets('#worlds2019', 2)

In [16]:
tweets[1].text

'@TeamGriffinLoL vs @G2esports tiebreaker highlights\U0001f973:\n\n3 min GRF FB\n18 min GRF 3 for 0 mid lane\n21 min GRF 4 for 2 blue side jungle\n27 min GRF ACE in base to win\n\n#lckwin #Worlds2019'